<a href="https://colab.research.google.com/github/skredenmathias/DS-Unit-2-Build/blob/master/Build_Week_Unit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading in datasets

In [1]:
import pandas as pd
# Remember to upload data to unit 2 build
!git clone https://github.com/skredenmathias/DS-Unit-1-Build.git
path = '/content/DS-Unit-1-Build/'

Cloning into 'DS-Unit-1-Build'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 35 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [2]:
year_2016 = pd.read_excel(path+'2016 complete match data OraclesElixir 2018-12-18.xlsx')
year_2016.shape

(19824, 84)

In [3]:
year_2017 = pd.read_excel(path+'2017matchdataOraclesElixir.xlsx')
year_2017.shape

(37404, 98)

In [4]:
spring_2018 = pd.read_excel(path+'2018-spring-match-data-OraclesElixir-2018-05-20.xlsx')
spring_2018.shape

(15900, 98)

In [5]:
summer_2018 = pd.read_excel(path+'2018 summer match data OraclesElixir 2018-09-27.xlsx')
summer_2018.shape

(16188, 98)

In [6]:
worlds_2018 = pd.read_excel(path+'2018-worlds-match-data-OraclesElixir-2018-11-03.xlsx')
worlds_2018.shape

(1428, 98)

In [7]:
spring_2019 = pd.read_excel(path+'2019-spring-match-data-OraclesElixir-2019-05-21.xlsx')
spring_2019.shape

(12864, 98)

In [8]:
summer_2019 = pd.read_excel(path+'2019-summer-match-data-OraclesElixir-2019-09-16.xlsx')
summer_2019.shape

(12612, 98)

In [12]:
worlds_2019 = pd.read_excel(path+'2019-summer-match-data-OraclesElixir-2019-11-10.xlsx')
worlds_2019.shape

(1428, 98)

# Merging dataframes

In [13]:
# Merging dataframes 1/2
dfs = [year_2017, spring_2018, summer_2018, worlds_2018, spring_2019, summer_2019, worlds_2019]
# Not including year_2016 due to several missing columns

# Concatenating
test = pd.concat(dfs)
test.shape

(97824, 98)

# Data Cleaning

In [14]:
def wrangle(X):
  X = X.copy()

  excess_columns = ['url','date','week', 'playerid','position','player','champion','ban1','ban2','ban3','ban4','ban5','k','d','a','doubles','triples','quadras','pentas','fb','fbassist', 'fbvictim', 'kpm','okpm',
                       'fd','heraldtime','dmgtochamps','dmgshare','earnedgoldshare','wards','wardshare','wardkills','visionwards','visionwardbuys','visiblewardclearrate','invisiblewardclearrate','totalgold','minionkills',
                       'monsterkills', 'monsterkillsownjungle','monsterkillsenemyjungle','goldat10','oppgoldat10','goldat15','oppgoldat15','xpat10','oppxpat10','csat10','oppcsat10','csat15','oppcsat15']
  X = X.drop(columns = excess_columns)
  return X

df = wrangle(test)
df.shape

(97824, 47)

In [15]:
# Removing excess drake columns, and HERALD

def wrangle2(X):
  X = X.copy()

  difficult_columns = ['elementals','oppelementals','firedrakes','waterdrakes',
                       'earthdrakes','airdrakes', 'herald']
  X = X.drop(columns = difficult_columns)
  return X

df = wrangle2(df)
df.shape

(97824, 40)

In [16]:
# Remove LPL games from dataframe, due to excess NaN values
LPL1 = df['league'] == 'LPL'
LPL = df[LPL1]

df = pd.concat([df, LPL]).drop_duplicates(keep=False)
df.shape

(75396, 40)

In [17]:
# Remove rows with NaN values
df = df.copy()
df = df.dropna()
df.shape

(72238, 40)

# Data Exploration

In [18]:
df.head(10)

,gameid,league,split,game,patchno,side,team,gamelength,result,teamkills,teamdeaths,fbtime,ckpm,fdtime,teamdragkills,oppdragkills,elders,oppelders,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochampsperminute,wpm,wcpm,earnedgpm,goldspent,gspd,cspm,gdat10,gdat15,xpdat10,csdat10,csdat15
0,1002300127,NALCS,2017-2,3,7.15,Blue,Team Liquid,25.383333,0,3.0,18.0,6.73213,0.827315,11.9418,0.0,2.0,0.0,0.0,1.0,12.368267,0.0,0,4.0,10.0,0.0,21.536467,0.0,1.0,343.729481,0.551543,0.078792,224.749048,9025.0,0.028090,9.258043,114.0,123.0,-8.0,5.0,14.0
1,1002300127,NALCS,2017-2,3,7.15,Blue,Team Liquid,25.383333,0,3.0,18.0,6.73213,0.827315,11.9418,0.0,2.0,0.0,0.0,1.0,12.368267,0.0,0,4.0,10.0,0.0,21.536467,0.0,1.0,169.402495,0.590939,0.118188,164.827840,6650.0,-0.241322,4.451740,53.0,-756.0,-27.0,0.0,-17.0
2,1002300127,NALCS,2017-2,3,7.15,Blue,Team Liquid,25.383333,0,3.0,18.0,6.73213,0.827315,11.9418,0.0,2.0,0.0,0.0,1.0,12.368267,0.0,0,4.0,10.0,0.0,21.536467,0.0,1.0,263.873933,0.630335,0.157584,241.177150,8700.0,-0.171789,9.809586,-22.0,-1050.0,-81.0,1.0,-1.0
3,1002300127,NALCS,2017-2,3,7.15,Blue,Team Liquid,25.383333,0,3.0,18.0,6.73213,0.827315,11.9418,0.0,2.0,0.0,0.0,1.0,12.368267,0.0,0,4.0,10.0,0.0,21.536467,0.0,1.0,408.417597,0.393959,0.039396,184.919764,8275.0,-0.191257,7.367039,-354.0,-1057.0,157.0,-7.0,-16.0
4,1002300127,NALCS,2017-2,3,7.15,Blue,Team Liquid,25.383333,0,3.0,18.0,6.73213,0.827315,11.9418,0.0,2.0,0.0,0.0,1.0,12.368267,0.0,0,4.0,10.0,0.0,21.536467,0.0,1.0,89.389363,0.827315,0.315167,120.822587,6880.0,-0.092862,0.472751,-133.0,-1114.0,-223.0,3.0,5.0
5,1002300127,NALCS,2017-2,3,7.15,Red,Dignitas,25.383333,1,18.0,3.0,6.73213,0.827315,11.9418,2.0,0.0,0.0,0.0,0.0,12.368267,1.0,1,10.0,4.0,1.0,21.536467,1.0,0.0,313.079448,0.551543,0.236376,298.577019,8775.0,-0.028090,9.218647,-114.0,-123.0,8.0,-5.0,-14.0
6,1002300127,NALCS,2017-2,3,7.15,Red,Dignitas,25.383333,1,18.0,3.0,6.73213,0.827315,11.9418,2.0,0.0,0.0,0.0,0.0,12.368267,1.0,1,10.0,4.0,1.0,21.536467,1.0,0.0,374.773473,0.630335,0.393959,256.462771,8475.0,0.241322,4.766907,-53.0,756.0,27.0,0.0,17.0
7,1002300127,NALCS,2017-2,3,7.15,Red,Dignitas,25.383333,1,18.0,3.0,6.73213,0.827315,11.9418,2.0,0.0,0.0,0.0,0.0,12.368267,1.0,1,10.0,4.0,1.0,21.536467,1.0,0.0,818.411031,0.275772,0.078792,356.489035,10335.0,0.171789,8.588313,22.0,1050.0,81.0,-1.0,1.0
8,1002300127,NALCS,2017-2,3,7.15,Red,Dignitas,25.383333,1,18.0,3.0,6.73213,0.827315,11.9418,2.0,0.0,0.0,0.0,0.0,12.368267,1.0,1,10.0,4.0,1.0,21.536467,1.0,0.0,539.290873,0.433355,0.118188,342.227708,10025.0,0.191257,9.179251,354.0,1057.0,-157.0,7.0,16.0
9,1002300127,NALCS,2017-2,3,7.15,Red,Dignitas,25.383333,1,18.0,3.0,6.73213,0.827315,11.9418,2.0,0.0,0.0,0.0,0.0,12.368267,1.0,1,10.0,4.0,1.0,21.536467,1.0,0.0,101.405121,1.339462,0.236376,242.162049,7550.0,0.092862,0.315167,133.0,1114.0,223.0,-3.0,-5.0


In [19]:
df['split'].value_counts()

2017-2      12048
2017-1      11760
2018-1      10296
2018-2       9142
2019-1       7752
2019-2       6552
2017-2po     1464
2018-2p      1452
2018-1P      1440
2017-W       1380
2019-W       1380
2017-1po     1344
2018-W       1344
2019-2P      1008
2017-2pr      900
2019-1P       864
2017-2rq      564
2018-2rq      540
2019-2RQ      528
2017-1pr      480
Name: split, dtype: int64

# Reducing dimensionality

## Grouping by 'gameid'

In [20]:
x = df.groupby(['gameid','team'])['result'].mean()
x = x.reset_index()
x.columns = ['gameid','team','avg_result']
x.head() 

,gameid,team,avg_result
0,136,Tempo Storm,1
1,136,eUnited,0
2,143,Delta Fox,1
3,143,Team Gates,0
4,144,Tempo Storm,0


In [21]:
df.columns

Index(['gameid', 'league', 'split', 'game', 'patchno', 'side', 'team',
       'gamelength', 'result', 'teamkills', 'teamdeaths', 'fbtime', 'ckpm',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elders', 'oppelders', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochampsperminute', 'wpm', 'wcpm', 'earnedgpm',
       'goldspent', 'gspd', 'cspm', 'gdat10', 'gdat15', 'xpdat10', 'csdat10',
       'csdat15'],
      dtype='object')

## Grouping by 'gameid', 'team', and 'split

In [0]:
# Here we have average values per team, per game.
# NOTE: We lose all non-numeric columns, some of which are intesesting:
# Note: added 'split' to groupby.
y = df.groupby(['gameid','team', 'split'])['league', 'game', 'patchno', 'side',
       'gamelength', 'result', 'teamkills', 'teamdeaths', 'fbtime', 'ckpm',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elders', 'oppelders', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochampsperminute', 'wpm', 'wcpm', 'earnedgpm',
       'goldspent', 'gspd', 'cspm', 'gdat10', 'gdat15', 'xpdat10', 'csdat10',
       'csdat15'].mean()
       
y = y.reset_index()
y.columns = ['gameid','team','split','patchno',
       'gamelength', 'result', 'teamkills', 'teamdeaths',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elders', 'oppelders', 'ft',
       'fttime', 'firstmidouter', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochampsperminute', 'wpm', 'wcpm', 'earnedgpm',
       'goldspent', 'gspd', 'cspm', 'gdat10', 'gdat15', 'xpdat10', 'csdat10',
       'csdat15']

In [23]:
print("y shape: ", y.shape) # 6 columns lost: league, game, side,
# fbtime, ckpm, firsttothreetowers
print('df shape: ', df.shape)
y.head()

y shape:  (12038, 34)
df shape:  (72238, 40)


,gameid,team,split,patchno,gamelength,result,teamkills,teamdeaths,fdtime,teamdragkills,oppdragkills,elders,oppelders,ft,fttime,firstmidouter,teamtowerkills,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochampsperminute,wpm,wcpm,earnedgpm,goldspent,gspd,cspm,gdat10,gdat15,xpdat10,csdat10,csdat15
0,136,Tempo Storm,2017-1,7.01,36.150000,1,14.0,8.0,11.853433,3.0,1.0,0.000000,0.000000,1.0,14.830800,0.0,10.0,4.0,1.0,31.511717,1.0,0.0,688.271093,1.005071,0.304288,413.178423,20025.000000,0.062620,10.677732,58.333333,259.000000,302.666667,7.000000,-2.333333
1,136,eUnited,2017-1,7.01,36.150000,0,8.0,14.0,11.853433,1.0,3.0,0.000000,0.000000,0.0,14.830800,1.0,4.0,10.0,0.0,31.511717,0.0,1.0,441.466113,0.903642,0.359613,328.392808,18833.333333,-0.062620,10.668511,-58.333333,-259.000000,-302.666667,-7.000000,2.333333
2,143,Delta Fox,2017-1,7.01,45.200000,1,23.0,17.0,16.040350,4.0,0.0,0.333333,0.000000,1.0,16.240767,1.0,9.0,2.0,0.0,28.586250,1.0,2.0,789.159292,0.892330,0.471976,435.343658,27423.333333,0.118755,10.324484,-173.000000,-73.666667,-128.333333,-5.666667,-7.666667
3,143,Team Gates,2017-1,7.01,45.200000,0,17.0,23.0,16.040350,0.0,4.0,0.000000,0.333333,0.0,16.240767,0.0,2.0,9.0,1.0,28.586250,2.0,1.0,699.705015,1.202065,0.265487,354.967552,24627.666667,-0.118755,10.309735,173.000000,73.666667,128.333333,5.666667,7.666667
4,144,Tempo Storm,2017-1,7.01,38.416667,0,2.0,15.0,18.030700,1.0,1.0,0.000000,0.000000,0.0,18.325433,0.0,2.0,11.0,0.0,27.266833,0.0,2.0,581.136659,1.093275,0.329718,312.971800,18944.333333,-0.140603,11.271150,310.000000,252.000000,442.000000,4.000000,-1.666667


## Removing 'gameid' -> possible final baseline dataframe

In [24]:
# Now it's getting interesting.

final_df1 = y.drop('gameid', axis=1)

final_df1 = final_df1.groupby(['team', 'split']).mean()
final_df1 = final_df1.reset_index()
final_df1.columns = ['team','split','patchno',
       'gamelength', 'result', 'teamkills', 'teamdeaths',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elders', 'oppelders', 'ft',
       'fttime', 'firstmidouter', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochampsperminute', 'wpm', 'wcpm', 'earnedgpm',
       'goldspent', 'gspd', 'cspm', 'gdat10', 'gdat15', 'xpdat10', 'csdat10',
       'csdat15']
print(final_df1.shape)
final_df1.head() # Consider removing gameid / groupby gameid also
# Here we see each team's averages per split.

(728, 33)


,team,split,patchno,gamelength,result,teamkills,teamdeaths,fdtime,teamdragkills,oppdragkills,elders,oppelders,ft,fttime,firstmidouter,teamtowerkills,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochampsperminute,wpm,wcpm,earnedgpm,goldspent,gspd,cspm,gdat10,gdat15,xpdat10,csdat10,csdat15
0,100 Thieves,2018-1,8.028947,39.105263,0.684211,10.105263,8.157895,11.788587,2.263158,1.789474,0.052632,0.140351,0.578947,13.556416,0.578947,7.578947,5.578947,0.578947,25.105712,1.263158,0.947368,615.379380,1.352520,0.601667,386.412152,21420.684211,-0.003062,11.550931,8.403509,28.052632,58.508772,-1.017544,-2.175439
1,100 Thieves,2018-1P,8.050000,36.977083,0.375000,7.625000,9.875000,7.107194,1.750000,2.625000,0.041667,0.041667,0.750000,13.254073,0.250000,5.000000,6.875000,0.250000,27.487513,0.500000,0.875000,544.369257,1.160463,0.550894,332.841965,19194.791667,-0.042006,11.368053,173.666667,-160.125000,114.333333,2.083333,1.708333
2,100 Thieves,2018-2,8.133333,37.967593,0.666667,14.500000,9.944444,12.313128,1.944444,1.944444,0.092593,0.074074,0.500000,13.218921,0.500000,8.000000,5.277778,0.555556,28.687762,1.055556,0.611111,634.886976,1.058557,0.508736,400.120252,21478.018519,0.062134,11.155886,-19.203704,113.444444,48.851852,-2.981481,-2.074074
3,100 Thieves,2018-2p,8.160000,35.258333,0.500000,9.416667,10.000000,9.191217,1.666667,2.416667,0.055556,0.027778,0.583333,13.672457,0.583333,6.000000,6.416667,0.416667,26.332015,0.583333,0.750000,590.769262,1.060018,0.540952,353.621034,18838.166667,-0.019128,11.133748,22.111111,-19.194444,38.194444,-2.111111,-6.527778
4,100 Thieves,2018-W,8.190000,32.116667,0.400000,13.400000,14.000000,12.993847,1.200000,1.400000,0.000000,0.000000,0.600000,12.686387,0.200000,5.000000,6.800000,0.400000,26.508837,0.600000,0.600000,601.255815,0.997673,0.428334,351.308287,16857.333333,-0.058251,10.685649,-67.666667,-326.466667,-164.200000,-4.666667,-8.266667


## Grouping by 'team' and 'patchno -> possible final baseline dataframe

In [25]:
# This could also be our final baseline groupby (minus results column)
y.groupby(['team', 'patchno']).mean()

gameid  gamelength  ...   csdat10   csdat15
team        patchno                            ...                    
100 Thieves 8.01     1.002440e+09   46.612500  ... -1.333333  0.916667
            8.02     1.002450e+09   40.370833  ... -0.166667 -0.750000
            8.03     1.002475e+09   38.154167  ... -4.333333 -6.750000
            8.04     1.002495e+09   32.325000  ...  1.250000 -4.416667
            8.05     1.002520e+09   37.178788  ...  1.424242  1.484848
...                           ...         ...  ...       ...       ...
paiN Gaming 9.11     1.138234e+06   35.454167  ...  2.416667 -1.666667
            9.12     1.091141e+06   36.625000  ...  2.666667  3.666667
            9.13     1.123573e+06   35.527778  ...  2.555556  1.611111
            9.14     1.155081e+06   34.361111  ...  2.444444 -2.666667
            9.15     1.102055e+06   27.050000  ... -4.666667 -9.333333

[2040 rows x 31 columns]

In [26]:
# Making the 'team' + 'patchno' into a dataframe
final_df2 = y.drop('gameid', axis=1)

final_df2 = final_df2.groupby(['team', 'patchno']).mean()
final_df2 = final_df2.reset_index()
final_df2.columns = ['team','patchno',
       'gamelength', 'result', 'teamkills', 'teamdeaths',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elders', 'oppelders', 'ft',
       'fttime', 'firstmidouter', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochampsperminute', 'wpm', 'wcpm', 'earnedgpm',
       'goldspent', 'gspd', 'cspm', 'gdat10', 'gdat15', 'xpdat10', 'csdat10',
       'csdat15']
print(final_df2.shape)
final_df2.head()

(2040, 32)


,team,patchno,gamelength,result,teamkills,teamdeaths,fdtime,teamdragkills,oppdragkills,elders,oppelders,ft,fttime,firstmidouter,teamtowerkills,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochampsperminute,wpm,wcpm,earnedgpm,goldspent,gspd,cspm,gdat10,gdat15,xpdat10,csdat10,csdat15
0,100 Thieves,8.01,46.612500,0.750000,10.750000,7.750000,14.337212,3.000000,1.500000,0.166667,0.250000,0.250000,13.210125,0.500000,7.750000,6.000000,0.500000,27.292417,1.250000,1.500000,668.497854,1.494992,0.807394,412.402995,26202.333333,0.014553,12.548556,-157.833333,-208.666667,-83.500000,-1.333333,0.916667
1,100 Thieves,8.02,40.370833,0.250000,5.000000,11.000000,11.635567,1.250000,3.000000,0.000000,0.166667,0.500000,13.839558,0.750000,5.000000,8.500000,0.000000,27.360529,0.250000,2.000000,530.228122,1.394491,0.645402,329.536565,20772.083333,-0.143042,11.622091,51.166667,113.916667,189.166667,-0.166667,-0.750000
2,100 Thieves,8.03,38.154167,0.750000,12.500000,8.250000,11.617554,2.500000,1.250000,0.083333,0.000000,0.500000,15.031683,0.750000,8.750000,5.250000,0.750000,23.297554,1.750000,0.750000,690.941710,1.499936,0.585288,397.903264,21555.666667,0.036463,11.057256,-102.166667,-316.166667,-91.666667,-4.333333,-6.750000
3,100 Thieves,8.04,32.325000,0.750000,9.750000,6.500000,12.110767,1.750000,1.500000,0.000000,0.083333,0.750000,11.989029,0.750000,7.500000,4.500000,0.750000,23.446104,1.250000,0.250000,546.837548,1.169377,0.469896,385.957155,17549.000000,0.027312,11.086526,141.666667,232.666667,186.083333,1.250000,-4.416667
4,100 Thieves,8.05,37.178788,0.545455,9.181818,9.090909,7.457845,2.090909,2.363636,0.030303,0.090909,0.818182,13.392991,0.181818,6.181818,5.818182,0.454545,26.483838,0.909091,0.636364,572.831030,1.158763,0.527901,354.669699,19657.727273,-0.012307,11.377672,165.242424,-3.181818,111.454545,1.424242,1.484848


# Fast first model

## Creating train/val/test sets

In [0]:
from sklearn.model_selection import train_test_split

In [30]:
train, test = train_test_split(final_df2, test_size = .20, random_state = 42) # stratify?
train.shape, test.shape

((1632, 32), (408, 32))